In [ ]:
"""
example of loading and plotting a large GNSS data file from Madrigal.
This example takes about 3 minutes total on a Apple with M2 CPU.

requires FFmpeg program
* macOS "brew install ffmpeg"
* Linux "apt install ffmpeg"
"""

from datetime import datetime, timedelta
from pathlib import Path

import h5py
import numpy as np

from matplotlib.pyplot import figure
import matplotlib.animation as anim

import cartopy.crs as ccrs

import mad_gnss_convert as mgc

In [ ]:
fn = Path("~/Downloads/los_20180506.001.h5.hdf5").expanduser().resolve(strict=True)

In [ ]:
print("loading times from {fn}, this may take a couple minutes...")

times = mgc.mad_times(fn)

print(
    f"{fn} contains data from {datetime.utcfromtimestamp(times[0])}"
    f" to {datetime.utcfromtimestamp(times[-1])}"
)

In [ ]:
treq = datetime(2018, 5, 6, 5, 20, 5)
atol = timedelta(seconds=3000)

# %% Filter by Time
start_time = treq - atol
end_time = treq + atol

time_index = mgc.time_range(times, start_time, end_time)

In [ ]:
del times

In [ ]:
with h5py.File(fn, "r") as f:
    gdlat = f["Data"]["Table Layout"]["gdlat"]
    glon = f["Data"]["Table Layout"]["glon"]
# < 2 minutes on Apple M2

lat_index = (gdlat >= -45) & (gdlat <= 45)
if lat_index.sum() == 0:
    print(f"gdlat: {gdlat} min: {gdlat.min()}  max: {gdlat.max()}")
    raise ValueError(f"No observations found in the requested latitude range.")

lon_index = (glon <= -120) & (glon >= -175)

if lon_index.sum() == 0:
    print(f"glon: {glon}  min: {glon.min()}  max: {glon.max()}")
    raise ValueError(f"No observations found in the requested longitude range.")

index = time_index & lat_index & lon_index
if index.sum() == 0:
    raise ValueError(
        f"No observations found in the requested time, latitude, longitude range."
    )

print(f"Found {index.sum()} good observations in {fn}.")

In [ ]:
del gdlat, glon

In [ ]:
with h5py.File(fn, "r") as f:
    working = f["Data"]["Table Layout"][index]

In [ ]:
unique_times = np.unique(working["ut1_unix"])
# these are sorted by numpy.unique
print(
    f"{unique_times.size} unique times found from {datetime.utcfromtimestamp(start_time)} to {datetime.utcfromtimestamp(end_time)}."
)

In [ ]:
print(
    f"{(working['tec'] < 0).sum()} negative TECu values found of {working['tec'].size} values."
)
vtec = working["tec"].clip(min=0)

In [ ]:
# %% Plotting
# takes about 30 seconds for a typical file parameter set on Apple M2

vfn = fn.with_suffix(".mp4")
print(f"Saving video to {vfn}")


cmap = "jet"

hfn = fn.with_name(fn.stem + "_histogram.mp4")
print(f"saving histogram to {hfn}")
fh = figure()

fg = figure(figsize=(16, 25))

vdpi = 100
# dots per inch, arbitrary choice for video resolution

vcodec = "h264"
# vcodec = "ffv1"
# ffv1 is lossless codec to avoid video compression artifacts
# if you get error:
# "Could not find tag for codec ffv1 in stream #0, codec not currently supported in container"
# then try codec='h264'

writer = anim.FFMpegWriter(fps=10, codec=vcodec)
writerh = anim.FFMpegWriter(fps=10, codec=vcodec)
# fps is frames / seconds, arbitrary choice for playback speed

pct = np.percentile(vtec, (1, 99))
# for mapping colorbar to reasonable range of values
vtec_min = vtec.min()
vtec_max = vtec.max()
print(f"TECu min {vtec_min}  max {vtec_max}")
print(f"percentiles TECu:  1 {pct[0]:.1f}   99 {pct[1]:.1f}")


with writer.saving(fg, vfn, dpi=vdpi) as w, writerh.saving(fh, hfn, dpi=vdpi) as wh:
    for j, t in enumerate(unique_times):
        i = working["ut1_unix"] == t
        v = vtec[i]
        lats = working[i]["gdlat"]
        lons = working[i]["glon"]

        ah = fh.gca()
        ah.hist(v, bins=100, range=(vtec_min, vtec_max))
        ah.set_ylim(0, 1200)  # arbitrary choice
        ah.set_xlabel("TECu")
        ah.set_ylabel("Count")

        ax = fg.add_subplot(projection=ccrs.PlateCarree())
        ax.text(
            -0.07,
            0.55,
            "Geodetic Latitude (deg)",
            va="bottom",
            ha="center",
            rotation="vertical",
            rotation_mode="anchor",
            transform=ax.transAxes,
        )
        # since cartopy takes over the axes, this is the only way you can label them
        ax.text(
            0.5,
            -0.05,
            "Geographic Longitude (deg)",
            va="bottom",
            ha="center",
            rotation="horizontal",
            rotation_mode="anchor",
            transform=ax.transAxes,
        )

        ttxt = f"VTEC measurement: {datetime.utcfromtimestamp(t).strftime('%Y-%m-%d %H:%M:%S')}"
        ax.set_title(ttxt)
        ah.set_title(ttxt)

        ax.set_extent([-175, -120, -45, 45])
        ax.coastlines()
        ax.gridlines(draw_labels=True)

        h = ax.scatter(lons, lats, c=v, vmin=pct[0], vmax=pct[1], cmap=cmap, s=25)
        # change s=25 to s=1 for smaller dots, etc

        cbar = fg.colorbar(h, ax=ax)
        cbar.ax.set_ylabel("TECU")

        pfn = f"{t}.png"
        print(f"Saving {t} {j/unique_times.size * 100:.1f} %", end="\r")

        # fg.savefig(pfn)
        w.grab_frame()

        wh.grab_frame()

        fg.clf()
        fh.clf()